In [39]:
from bs4 import BeautifulSoup
import urllib.request
import re

In [44]:
import pandas as pd

In [2]:
fp = urllib.request.urlopen("http://www.elections.ny.gov/ENR/NYSENRAccessible.html")
mybytes = fp.read()

mystr = mybytes.decode("utf8")
fp.close()

In [3]:
len(mystr)

962853

In [8]:
soup = BeautifulSoup(mystr, 'html.parser')

In [50]:
info = [{
    "h4": h.text.strip(),
    "p": h.find_next("p").text.strip(),
    "table": h.find_next("table").text.strip(),
    "rows": [[j.text.strip() for j in i.find_all("td")] for i in h.find_next("table").find_all("tr")]} for h in soup.find_all("h4")]

In [51]:
# all h4 values
set([i["h4"] for i in info])

{'Attorney General\xa0DEM Primary',
 'Attorney General\xa0REF Primary',
 'Governor\xa0DEM Primary',
 'Lt. Governor\xa0DEM Primary',
 'State Assembly\xa0CON Primary > 118th Assembly District',
 'State Assembly\xa0CON Primary > 142nd Assembly District',
 'State Assembly\xa0CON Primary > 62nd Assembly District',
 'State Assembly\xa0DEM Primary > 104th Assembly District',
 'State Assembly\xa0DEM Primary > 107th Assembly District',
 'State Assembly\xa0DEM Primary > 121st Assembly District',
 'State Assembly\xa0DEM Primary > 124th Assembly District',
 'State Assembly\xa0DEM Primary > 136th Assembly District',
 'State Assembly\xa0DEM Primary > 137th Assembly District',
 'State Assembly\xa0DEM Primary > 18th Assembly District',
 'State Assembly\xa0DEM Primary > 20th Assembly District',
 'State Assembly\xa0DEM Primary > 30th Assembly District',
 'State Assembly\xa0DEM Primary > 33rd Assembly District',
 'State Assembly\xa0DEM Primary > 39th Assembly District',
 'State Assembly\xa0DEM Primary > 

In [133]:
ag_info = [i for i in info if i["h4"] == "Attorney General\xa0DEM Primary"]

In [134]:
prep_ag_info = [[i["h4"], i["p"], i["table"]] + r for i in ag_info for r in i["rows"][1:]]

In [135]:
ag_df = pd.DataFrame(prep_ag_info, columns = ["h4", "p", "table", "candidate", "party", "ttl", "pcnt"])

In [136]:
reg_ttl_and_county = (ag_df
    .p
    .str.replace("\r", "")
    .str.replace("\n", "")
    .str.findall(".*Active Registered Voters (\d*) .* ([A-Za-z.]*) (?:(?:county)|(?:Counties))", re.MULTILINE))

In [137]:
ag_df["ttl_reg_voters"] = reg_ttl_and_county.map(lambda x: x[0][0])
ag_df["county"] = reg_ttl_and_county.map(lambda x: x[0][1])

In [138]:
ag_df

,h4,p,table,candidate,party,ttl,pcnt,ttl_reg_voters,county
0,Attorney General DEM Primary,Vote for 1 Active Registered V...,Candidate Name\nParty\nTotal Votes\nVotes Perc...,Sean Patrick Maloney,DEM,357462,23.85,5621811,All
1,Attorney General DEM Primary,Vote for 1 Active Registered V...,Candidate Name\nParty\nTotal Votes\nVotes Perc...,Letitia A. James,DEM,579298,38.66,5621811,All
2,Attorney General DEM Primary,Vote for 1 Active Registered V...,Candidate Name\nParty\nTotal Votes\nVotes Perc...,Leecia R. Eve,DEM,48738,3.25,5621811,All
3,Attorney General DEM Primary,Vote for 1 Active Registered V...,Candidate Name\nParty\nTotal Votes\nVotes Perc...,Zephyr Teachout,DEM,442114,29.50,5621811,All
4,Attorney General DEM Primary,Vote for 1 Active Registered V...,Candidate Name\nParty\nTotal Votes\nVotes Perc...,Blank,,58979,3.94,5621811,All
5,Attorney General DEM Primary,Vote for 1 Active Registered V...,Candidate Name\nParty\nTotal Votes\nVotes Perc...,Void,,9003,0.60,5621811,All
6,Attorney General DEM Primary,Vote for 1 Active Registered V...,Candidate Name\nParty\nTotal Votes\nVotes Perc...,Write-in,,2968,0.20,5621811,All
7,Attorney General DEM Primary,Vote for 1 Active Registered V...,Candidate Name\nParty\nTotal Votes\nVotes Perc...,Sean Patrick Maloney,DEM,8513,31.47,91122,Albany
8,Attorney General DEM Primary,Vote for 1 Active Registered V...,Candidate Name\nParty\nTotal Votes\nVotes Perc...,Letitia A. James,DEM,3952,14.61,91122,Albany
9,Attorney General DEM Primary,Vote for 1 Active Registered V...,Candidate Name\nParty\nTotal Votes\nVotes Perc...,Leecia R. Eve,DEM,1600,5.92,91122,Albany
